## Reddit scraper

- reddit ranks its subreddits by total member count - here are [the top 250 subreddits ranked by ths metric](https://www.reddit.com/best/communities/1/)

- at the top of every subreddit you will find two numbers: 

    - total member count: the total number of registered reddit users who have joined the subreddit
    - online now: number of registered reddit users who have joined the subreddit and who have been on the subreddit [sometime in the past 15 minutes](https://www.reddit.com/r/NoStupidQuestions/comments/9pn1ib/when_a_subbreddit_has_the_online_count_is_that/)

- since reddit has no identity verification on registration, both numbers likely contain "bot" accounts, and the former long un-used accounts

- for example, here are screenshots of the top 3 subreddits by size - displaying these two numbers


<img src="imgs/funny.png" width=25%>
<img src="imgs/ask_reddit.png" width=25%>
<img src="imgs/gaming.png" width=25%>

- lets see how we can scrape these two numbers from the top 50 subreddits as ranked by reddit - by total member count

- we will also setup a simple schedule to continuously scrape these numbers ever interval of our choosing (e.g., every 30 minutes)

## Building the scraping library

- we'll call our library `reddit_scraper`
- in this library we build logic for
    - instantiating a sqlite database to store data
    - handling top subreddit links
    - making requests to these links to collect their html
    - parsing each html page to sieve out our two numbers of interest
    - writing these two numbers in a `sqlite` database

## Harvesting links

- since reddit provides [the top 250 subreddits ranked by total member count](https://www.reddit.com/best/communities/1/) we can just copy and paste links to the top 50
- each will contain the two metrics we're looking to record
- we record these links in `reddit_scraper/links`
- we can view these by using the jupyter magic command `??` as done below

In [6]:
from reddit_scraper.links import links??

Type:        module
String form: <module 'reddit_scraper.links' from '/Users/jeremywatt/Desktop/content_creation/reddit_scraper/reddit_scraper/links.py'>
File:        ~/Desktop/content_creation/reddit_scraper/reddit_scraper/links.py
Source:     
links = [
    "https://www.reddit.com/r/funny/",
    "https://www.reddit.com/r/AskReddit/",
    "https://www.reddit.com/r/gaming/",
    "https://www.reddit.com/r/worldnews/",
    "https://www.reddit.com/r/todayilearned/",
    "https://www.reddit.com/r/aww/",
    "https://www.reddit.com/r/Music/",
    "https://www.reddit.com/r/movies/",
    "https://www.reddit.com/r/science/",
    "https://www.reddit.com/r/memes/",
    "https://www.reddit.com/r/Showerthoughts/",
    "https://www.reddit.com/r/pics/",
    "https://www.reddit.com/r/Jokes/",
    "https://www.reddit.com/r/news/",
    "https://www.reddit.com/r/videos/",
    "https://www.reddit.com/r/space/",
    "https://www.reddit.com/r/askscience/",
    "https://www.reddit.com/r/DIY",
    "https://w

## Creating a sqlite database to store data

- sqlite is the most popular database on the planet
- the database is just a simple file
- out of the box python comes with apis to create, insert, delete, and query data with sqlite
- lets look at how to create a sqlite database using basic python commands
- here we instantiate the database with four columns
    - reddit_url: the url of the subreddit we scrape
    - timestamp: of when we scrape
    - total_member_count: total count of members listed on the page
    - total_online_count: total count of members online listed on the page

In [17]:
from reddit_scraper.sqlite_interactions import create_sqlite_db??

Signature: create_sqlite_db(db_filepath: str = 'reddit_scraper.db') -> None
Docstring: <no docstring>
Source:   
def create_sqlite_db(db_filepath: str = "reddit_scraper.db") -> None:
    try:
        # Connect and create cursor
        conn = sqlite3.connect(db_filepath)
        cursor = conn.cursor()

        # Create the table  - delete old table if it exists
        cursor.execute("DROP TABLE IF EXISTS reddit_scraper")

        # Create the table
        query = """
            CREATE TABLE IF NOT EXISTS reddit_scraper (
                reddit_url TEXT,
                timestamp INTEGER,
                total_member_count INTEGER,
                total_online_count INTEGER
            );
        """
        cursor.execute(query)

        # Create index on reddit_url for faster retrieval
        query = "CREATE INDEX index_reddit_url ON reddit_scraper (reddit_url);"
        query = "CREATE INDEX index_reddit_timestamp ON reddit_scraper (timestamp);"
        cursor.execute(query)

   

## Making requests

- because we want to scrape 50 subreddits we don't want to make requests in sequence, as we will need to wait for one request to end to start tne next
- we want to make the requests async to avoid having to wait
- to do this we use [grequests](https://github.com/spyoungtech/grequests)
- the api for doing this looks very similar to the common [requests](https://github.com/psf/requests) api
- be careful not to run this too many times, too fast, to avoid bans

In [21]:
from reddit_scraper.make_requests import make_requests??

Signature: make_requests(links: list) -> list
Docstring: <no docstring>
Source:   
def make_requests(links: list) -> list:
    try:
        # "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36"
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36"
        }
        rs = (grequests.get(u, headers=headers) for u in links)

        def exception_handler(request, exception):
            print(exception)

        responses = grequests.map(rs, exception_handler=exception_handler)
        for ind, resp in enumerate(responses):
            if resp.status_code != 200:
                print(
                    f"FAILURE: link {links[ind]} responded with status_code {resp.status_code}"
                )
            else:
                print(
                    f"SUCCESS: link {links[ind]} responded wi

## Parsing request responses

- for html parsing we'll use the popular [beautifulsoup](https://beautiful-soup-4.readthedocs.io/en/latest/)
- because we want to parse out just two numbers - and those are placed within the DOM in the exact same manner on all subreddits - our use is very simple

In [23]:
from reddit_scraper.parser import parse_responses??

Signature: parse_responses(responses: list, links: list) -> list
Docstring: <no docstring>
Source:   
def parse_responses(responses: list, links: list) -> list:
    try:
        created_at = int(datetime.now(tz=timezone.utc).timestamp())
        data = []
        for ind, response in enumerate(responses):
            if response is not None:
                # make request
                text = response.text
                soup = BeautifulSoup(text, features="html.parser")

                # collect total member and online counts
                members_span = soup.select_one('span:-soup-contains("members")')
                total_member_count = int(
                    members_span.findChild("faceplate-number")["number"]
                )

                online_span = soup.select_one('span:-soup-contains("online")')
                total_online_count = int(
                    online_span.findChild("faceplate-number")["number"]
                )

                # create entry
     

## Writing parsed responses to disk

- we then write our simple parsed response for each subreddit to disk - to the sqlite database we made previously

In [26]:
from reddit_scraper.sqlite_interactions import put_rows??

Signature: put_rows(rows_to_write: list, db_filepath: str = 'reddit_scraper.db') -> None
Docstring: <no docstring>
Source:   
def put_rows(rows_to_write: list, db_filepath: str="reddit_scraper.db") -> None:
    # Connect and create cursor
    conn = sqlite3.connect(db_filepath)
    cursor = conn.cursor()

    # Setup query
    query = "INSERT INTO reddit_scraper (reddit_url, timestamp, total_member_count, total_online_count) VALUES (?, ?, ?, ?)"
    
    # Insert data into the table
    for entry in rows_to_write:
        reddit_url = entry["reddit_url"]
        timestamp = entry["timestamp"]
        total_member_count = entry["total_member_count"]
        total_online_count = entry["total_online_count"]

        cursor.execute(
            query,
            (reddit_url, timestamp, total_member_count, total_online_count),
        )

    conn.commit()
    conn.close()
File:      ~/Desktop/content_creation/reddit_scraper/reddit_scraper/sqlite_interactions.py
Type:      function

## Scheduling scraping

- we can schedule regular scrapes of this data on an interval using [schedule](https://schedule.readthedocs.io/en/stable/)
- to use schedule we simply decorate our main code - that makes the requests, parses the responses, and writes the results to our database - with a simple decorator
- to execute on a 30 minute interval use the decorator `@repeat(every(30).minutes)`

In [28]:
from reddit_scraper.main import main??

Signature: main() -> None
Docstring: <no docstring>
Source:   
@repeat(every(30).minutes)
def main() -> None:
    try:
        responses = make_requests(links)
        data = parse_responses(responses, links)
        put_rows(data)
        print("SUCCESS: main finished successfully")
    except Exception as e:
        print(f"FAILURE: main failed with exception {e}")
        raise e
File:      ~/Desktop/content_creation/reddit_scraper/reddit_scraper/main.py
Type:      function

## Viewing the data in a [dash](https://dash.plotly.com/) dashboard

- to visualize the scraped data open a new terminal and type

```python
python dashboard/main.py
```

- of course to see any meaningful trends you will need to let the scraper run for a while

<img src="imgs/reddit_dashboard.png" width=100%>